In [14]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dask.distributed import Client
import glob
import codecs
import cf_units
import netCDF4 as nc

In [15]:
try:
    c
except:
    import os
    c = Client(n_workers=int(os.getenv("PBS_NCPUS")), threads_per_worker=1)
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 2
Total threads: 2,Total memory: 9.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34159,Workers: 2
Dashboard: /proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 9.00 GiB
Comm: tcp://127.0.0.1:46815,Total threads: 1
Dashboard: /proxy/43221/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:40561,


In [16]:
PLUMBER2_path      = "/g/data/w97/mm3972/data/PLUMBER2/"
PLUMBER2_flux_path = "/g/data/w97/mm3972/data/Fluxnet_data/Post-processed_PLUMBER2_outputs/Nc_files/Flux/"
PLUMBER2_met_path  = "/g/data/w97/mm3972/data/Fluxnet_data/Post-processed_PLUMBER2_outputs/Nc_files/Met/"

## Get site names 

In [17]:
site_names     = [os.path.basename(site_path).split("_")[0] for site_path in all_site_path]
all_site_path  = sorted(glob.glob(PLUMBER2_met_path+"/*.nc"))

NameError: name 'all_site_path' is not defined

In [27]:
var_name       = 'latent'  #'trans'  #
bin_by         = 'EF_obs' #'EF_model' #'EF_obs'#

## Read IGBP Vegetation Types from files

In [28]:
def get_veg_short(site_name:str) -> str:
    file_name = glob.glob(f"{PLUMBER2_met_path}/*{site_name}*.nc")[0]
    return codecs.decode(xr.open_dataset(file_name).IGBP_veg_short.data, 'utf-8').strip()

IGBP_dict = {
    site_name: get_veg_short(site_name)
    for site_name in site_names
}

## Read Lat/Lon Grids

In [29]:
def get_lat_lon(sn: str):
    filename = glob.glob(f"{PLUMBER2_met_path}/*{sn}*nc")[0]
    f = xr.open_dataset(filename)
    return {'latitude': f.latitude[0,0].item(), 'longitude': f.longitude[0,0].item()}
lat_lon = {sn: get_lat_lon(sn) for sn in site_names}
lat_dict = {sn: lat_lon[sn]['latitude'] for sn in site_names}
lon_dict = {sn: lat_lon[sn]['longitude'] for sn in site_names}

## Read Climate Class

In [30]:
climate_class_path = '/g/data/w97/mm3972/data/Köppen-Geiger_climate_classification/Beck_KG_V1/Beck_KG_V1_present_0p0083.nc'
f = xr.open_dataset(climate_class_path)

In [31]:
def get_climate_class(lat, lon):
    climate_class = int(f.climate_class.sel(latitude=lat, longitude=lon, method='nearest').item()) - 1
    return f.class_name[climate_class].item()

In [32]:
clim_class_dict = {
    sn: get_climate_class(lat_dict[sn], lon_dict[sn])
    for sn in site_names
}

In [33]:
## Read data function

In [34]:
def read_data(var_name, site_name, input_file, bin_by=None):
    f = xr.open_dataset(input_file, use_cftime=False, decode_times=False)
    
    model_in_list = f[f"{var_name}_models"].values
    time = nc.num2date(
        f['CABLE_time'].values, f['CABLE_time'].attrs['units'],
        only_use_cftime_datetimes=False,
        only_use_python_datetimes=True,
    )
    ntime=len(time)

    # Create accumulators
    var_output = pd.DataFrame(time, columns=['time'])
    model_out_list = []

    if bin_by=='EF_model':
        model_in_list  = f['EF_models'].values

    for model_in in model_in_list:
        # Set the var and time names of the model 
        model_var_name = f"{model_in}_{var_name}"
        model_time_name = f"{model_in}_time"
        model_ntime = len(f[model_time_name])

        # If the model has full time series
        if model_ntime == ntime:
            model_out_list.append(model_in)
            if var_name == "trans":
                var_output[model_in] = f[model_var_name].values * 3600
            else:
                var_output[model_in] = f[model_var_name].values
            if bin_by == "EF_model":
                model_bin_name = f"{model_in}_EF"
                var_output[f"{model_in}_EF"] = f[model_bin_name].values
            
    if var_name == 'latent' or var_name == 'sensible':
        var_output['obs'] = f[f"obs_{var_name}"].values
        model_out_list.append('obs')

    if bin_by=='EF_obs' or bin_by=='EF_model' :
        var_output['obs_EF'] = f["obs_EF"].values
        
    # Read VPD and soil moisture information
    var_output['VPD']      = f['VPD'].values
    var_output['obs_Tair'] = f['obs_Tair'].values
    var_output['obs_Qair'] = f['obs_Qair'].values

    ntime      = len(var_output)
    month      = np.zeros(ntime)
    hour       = np.zeros(ntime)
    # site       = np.full([ntime], site_name.rjust(6), dtype=str)

    for i in np.arange(ntime):
        month[i] = var_output['time'][i].month
        hour[i]  = var_output['time'][i].hour

    var_output['month']     = month
    var_output['hour']      = hour
    var_output['site_name'] = site_name

    return var_output, model_out_list

i, site_name = 0, site_names[0]
input_file = f"/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/nc_files/{site_name}.nc"
read_data(var_name, site_name, input_file, bin_by)

(                     time     ACASA      CABLE  CABLE-POP-CN  \
 0     2010-01-01 00:30:00 -5.689844  32.191139     55.521278   
 1     2010-01-01 01:00:00 -4.842229  27.883013     44.957954   
 2     2010-01-01 01:30:00 -4.473716  20.909231     52.848763   
 3     2010-01-01 02:00:00 -4.925210  48.849258     89.795792   
 4     2010-01-01 02:30:00 -5.239734  64.710876    135.119995   
 ...                   ...       ...        ...           ...   
 17515 2010-12-31 22:00:00 -5.876469  17.854084     24.210634   
 17516 2010-12-31 22:30:00 -5.504390  15.296217     22.639807   
 17517 2010-12-31 23:00:00 -4.853248  11.720897     18.979084   
 17518 2010-12-31 23:30:00 -3.530991   7.094285     14.547200   
 17519 2011-01-01 00:00:00 -3.045460   5.313951     12.953535   
 
        CHTESSEL_Ref_exp1  CHTESSEL_ERA5_3      CLM5a       GFDL  JULES_GL9  \
 0              23.719130        14.213023  12.440999  56.147644  44.504723   
 1              23.719130        14.213023   9.597205  50.98

## Read all sites data

In [36]:
%%time
for i, site_name in enumerate(site_names):
    
    print(f"{i:4}/{len(site_names)} - {site_name}")
    
    input_file = "/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/nc_files/"+site_name+".nc"
    var_output_tmp, model_out_list = read_data(var_name, site_name, input_file, bin_by)

    # Add veg type
    var_output_tmp['IGBP_type']    = IGBP_dict[site_name]

    # Add climate type
    var_output_tmp['climate_type'] = clim_class_dict[site_name]

    if i == 0:
        var_output         = var_output_tmp
        pre_model_out_list = model_out_list
    else:
        if pre_model_out_list != model_out_list: 
            # Find the elements that are only in pre_model_out_list
            only_in_pre_model_out_list = np.setdiff1d(pre_model_out_list, model_out_list, assume_unique=True)
            print('Elements only in pre_model_out_list:',only_in_pre_model_out_list)
            # Set the missing model simulation as np.nan
            for missed_model in only_in_pre_model_out_list:
                var_output_tmp[missed_model] = np.nan
                if bin_by=='EF_model':        
                    var_output_tmp[missed_model+'_EF'] = np.nan

        # connect different sites data together
        var_output = var_output.append(var_output_tmp, ignore_index=True)

    var_output_tmp=None
    
model_out_list = pre_model_out_list

   0/170 - AR-SLu
   1/170 - AT-Neu
   2/170 - AU-ASM
   3/170 - AU-Cow
   4/170 - AU-Cpr
   5/170 - AU-Ctr
   6/170 - AU-Cum
   7/170 - AU-DaP
   8/170 - AU-DaS
   9/170 - AU-Dry
  10/170 - AU-Emr
  11/170 - AU-GWW
  12/170 - AU-Gin
  13/170 - AU-How
  14/170 - AU-Lit
  15/170 - AU-Otw
  16/170 - AU-Rig
  17/170 - AU-Rob
Elements only in pre_model_out_list: ['ORCHIDEE_tag2.1' 'ORCHIDEE_tag3_2']
  18/170 - AU-Sam
  19/170 - AU-Stp
  20/170 - AU-TTE
  21/170 - AU-Tum
  22/170 - AU-Whr
  23/170 - AU-Wrr
Elements only in pre_model_out_list: ['ORCHIDEE_tag2.1' 'ORCHIDEE_tag3_2']
  24/170 - AU-Ync
Elements only in pre_model_out_list: ['QUINCY']
  25/170 - BE-Bra
  26/170 - BE-Lon
  27/170 - BE-Vie
  28/170 - BR-Sa3
  29/170 - BW-Ma1
  30/170 - CA-NS1
  31/170 - CA-NS2
Elements only in pre_model_out_list: ['ORCHIDEE_tag2.1' 'ORCHIDEE_tag3_2']
  32/170 - CA-NS4
  33/170 - CA-NS5
  34/170 - CA-NS6
Elements only in pre_model_out_list: ['MuSICA']
  35/170 - CA-NS7
Elements only in pre_model_out_

In [37]:
var_output.to_csv(f'/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/{var_name}_all_sites.csv')

In [39]:
var_output

,time,ACASA,CABLE,CABLE-POP-CN,CHTESSEL_Ref_exp1,CHTESSEL_ERA5_3,CLM5a,GFDL,JULES_GL9,MATSIRO,...,obs,obs_EF,VPD,obs_Tair,obs_Qair,month,hour,site_name,IGBP_type,climate_type
0,2010-01-01 00:30:00,-5.689844,32.191139,55.521278,23.719130,14.213023,12.440999,56.147644,44.504723,28.654470,...,116.800003,NaN,1.2924,292.649994,0.006580,1.0,0.0,AR-SLu,MF,BSk
1,2010-01-01 01:00:00,-4.842229,27.883013,44.957954,23.719130,14.213023,9.597205,50.984962,30.489576,21.738682,...,29.816200,NaN,0.8980,291.850006,0.008465,1.0,1.0,AR-SLu,MF,BSk
2,2010-01-01 01:30:00,-4.473716,20.909231,52.848763,22.672579,11.441057,25.509281,51.721157,26.630692,21.010851,...,12.800000,NaN,0.9025,291.450012,0.008073,1.0,1.0,AR-SLu,MF,BSk
3,2010-01-01 02:00:00,-4.925210,48.849258,89.795792,52.327900,11.655688,34.689869,72.450790,14.385949,70.327156,...,30.003599,NaN,0.9070,291.549988,0.008145,1.0,2.0,AR-SLu,MF,BSk
4,2010-01-01 02:30:00,-5.239734,64.710876,135.119995,104.630157,13.281853,38.612492,90.080818,8.263578,119.643463,...,164.699997,NaN,1.0735,291.950012,0.007412,1.0,2.0,AR-SLu,MF,BSk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17137987,2008-12-31 22:00:00,-3.434519,0.398925,-1.219558,0.452838,7.881705,0.108744,0.156870,0.039569,-3.808859,...,8.872780,NaN,0.6075,295.750000,0.015418,12.0,22.0,ZM-Mon,DBF,Cwa
17137988,2008-12-31 22:30:00,-3.237316,0.345242,-1.610697,0.373772,5.402277,-11.254688,0.156763,0.033414,-6.746114,...,15.000700,NaN,0.5068,295.147003,0.015416,12.0,22.0,ZM-Mon,DBF,Cwa
17137989,2008-12-31 23:00:00,-2.699634,0.243988,-2.162361,0.277473,5.950359,-8.518088,0.156656,0.024451,-10.615149,...,5.107330,NaN,0.3987,294.627014,0.015573,12.0,23.0,ZM-Mon,DBF,Cwa
17137990,2008-12-31 23:30:00,-0.707895,0.238878,-2.365701,0.311701,11.926942,-9.626886,0.156549,0.018539,-11.128715,...,3.878990,NaN,0.3522,294.527008,0.015785,12.0,23.0,ZM-Mon,DBF,Cwa


## Plotting

In [44]:
def plot_spatial_land_days(var_name, site_names, PLUMBER2_met_path, bin_by=None, bin_vals=None, message=None, day_time=False, summer_time=False, IGBP_type=None, clim_type=None):

    # read the data
    var_output    = pd.read_csv(f'/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/{var_name}_all_sites.csv')

    # get the model namelist
    f             = nc.Dataset("/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/nc_files/AR-SLu.nc", mode='r')
    model_in_list = f.variables[var_name + '_models']
    ntime         = len(f.variables['CABLE_time'])
    model_out_list= []

    for model_in in model_in_list:
        if len(f.variables[f"{model_in}_time"]) == ntime:
            model_out_list.append(model_in)
    if var_name in ['latent','sensible']:
        model_out_list.append('obs')

    # Select periods
    if day_time:
        day_mask    = (var_output['hour'] >= 9) & (var_output['hour'] <= 16)
        var_output     = var_output[day_mask]

    if summer_time:
        summer_mask = (var_output['month'] > 11) | (var_output['month']< 3)
        var_output  = var_output[summer_mask]

    if IGBP_type!=None:
        IGBP_mask   = (var_output['IGBP_type'] == IGBP_type)
        var_output  = var_output[IGBP_mask]
    
    if clim_type!=None:
        clim_mask   = (var_output['climate_type'] == clim_type)
        var_output  = var_output[clim_mask]

    # if using observed EF
    if bin_by == 'EF_obs':
        # exclude the time steps, Qh<0 or Qle+Qh<10.
        EF_notNan_mask = ~ np.isnan(var_output['obs_EF'])
        var_output     = var_output[EF_notNan_mask]

        # select the data
        if len(bin_vals) == 1:
            EF_mask  = (var_output['obs_EF'] <= bin_vals[0]) 
        elif len(bin_vals) == 2:
            EF_mask  = (var_output['obs_EF'] >= bin_vals[0]) & (var_output['obs_EF'] <= bin_vals[1]) 

        # mask out the time steps that obs_EF is np.nan
        var_output  = var_output[EF_mask]

        # free memory
        EF_notNan_mask=None

    # if using model simulated EF
    if bin_by == 'EF_model': 
        # go throught all models which have the var_name
        for model_out_name in model_out_list:
            # bin the variable by the model's own EF
            if len(bin_vals) == 1:
                var_output[model_out_name] = np.where(
                    (var_output[model_out_name+'_EF'] <= bin_vals[0]),
                    var_output[model_out_name], np.nan )
            elif len(bin_vals) == 2:
                var_output[model_out_name] = np.where(
                    (var_output[model_out_name+'_EF'] >= bin_vals[0]) 
                  & (var_output[model_out_name+'_EF'] <= bin_vals[1]),
                    var_output[model_out_name], np.nan )

    # ============ Setting for plotting ============
    cmap     = plt.cm.rainbow #YlOrBr #coolwarm_r

    fig, ax  = plt.subplots(nrows=2, ncols=1, figsize=[10,15],sharex=True, sharey=False, squeeze=True) #
    # fig, ax = plt.subplots(figsize=[10, 7])
    # plt.subplots_adjust(wspace=0.0, hspace=0.0)

    plt.rcParams['text.usetex']     = False
    plt.rcParams['font.family']     = "sans-serif"
    plt.rcParams['font.serif']      = "Helvetica"
    plt.rcParams['axes.linewidth']  = 1.5
    plt.rcParams['axes.labelsize']  = 14
    plt.rcParams['font.size']       = 14
    plt.rcParams['legend.fontsize'] = 14
    plt.rcParams['xtick.labelsize'] = 14
    plt.rcParams['ytick.labelsize'] = 14

    almost_black = '#262626'
    # change the tick colors also to the almost black
    plt.rcParams['ytick.color']     = almost_black
    plt.rcParams['xtick.color']     = almost_black

    # change the text colors also to the almost black
    plt.rcParams['text.color']      = almost_black

    # Change the default axis colors from black to a slightly lighter black,
    # and a little thinner (0.5 instead of 1)
    plt.rcParams['axes.edgecolor']  = almost_black
    plt.rcParams['axes.labelcolor'] = almost_black
    
    # Set the colors for different models
    model_colors = set_model_colors()

    props = dict(boxstyle="round", facecolor='white', alpha=0.0, ec='white')

    # Set variable needed to plot
    var_plot    = var_output #var_sm_mid #var_summur

    # Set up the VPD bins
    vpd_top     = 6.54
    vpd_bot     = 0.02
    vpd_interval= 0.04
    vpd_series  = np.arange(vpd_bot,vpd_top,vpd_interval)

    # Set up the values need to draw
    vpd_sum     = len(vpd_series)
    var_vals    = np.zeros((len(model_out_list), vpd_sum))
    var_vals_top= np.zeros((len(model_out_list), vpd_sum))
    var_vals_bot= np.zeros((len(model_out_list), vpd_sum))

    # Binned by VPD
    for j, vpd_val in enumerate(vpd_series):
        mask_vpd       = (var_plot['VPD'] > vpd_val-0.02) & (var_plot['VPD'] < vpd_val+0.02)
        try:
            var_masked = var_plot[mask_vpd]
        except:
            var_masked = np.nan

        # Draw the line for different models
        for i, model_out_name in enumerate(model_out_list):

            # calculate mean value
            var_vals[i,j] = var_masked[model_out_name].mean(skipna=True)

            if 0:
                # using 1 std as the uncertainty
                var_std   = var_masked[model_out_name].std(skipna=True)
                print(i,j,var_vals[i,j],var_std)
                var_vals_top[i,j] = var_vals[i,j] + var_std
                var_vals_bot[i,j] = var_vals[i,j] - var_std

            if 1:
                # using percentile as the uncertainty
                var_temp  = var_masked[model_out_name]
                mask_temp = ~ np.isnan(var_temp)
                if np.any(mask_temp):
                    var_vals_top[i,j] = np.percentile(var_temp[mask_temp], 75)
                    var_vals_bot[i,j] = np.percentile(var_temp[mask_temp], 25)
                else:
                    var_vals_top[i,j] = np.nan
                    var_vals_bot[i,j] = np.nan

    # Plot the PDF of the normal distribution
    hist = ax[0].hist(var_plot['VPD'], bins=400, density=False, alpha=0.6, color='g', histtype='stepfilled')
    
    for i, model_out_name in enumerate(model_out_list):
        # Calculate uncertainty
        if i == 0:
            df_var_vals     = pd.DataFrame({model_out_name: var_vals[i,:]})
            df_var_vals_top = pd.DataFrame({model_out_name: var_vals_top[i,:]})
            df_var_vals_bot = pd.DataFrame({model_out_name: var_vals_bot[i,:]})
        else:
            df_var_vals[model_out_name]    = var_vals[i,:]
            df_var_vals_top[model_out_name]= var_vals_top[i,:]
            df_var_vals_bot[model_out_name]= var_vals_bot[i,:]

        line_color = model_colors[model_out_name] #plt.cm.tab20(i / len(model_out_list))

        plot = ax[1].plot(vpd_series, df_var_vals[model_out_name], lw=2.0,  
                          color=line_color, alpha=0.9, label=model_out_name) #edgecolor='none', c='red' .rolling(window=10).mean()
        fill = ax[1].fill_between(vpd_series, df_var_vals_bot[model_out_name],
                               df_var_vals_top[model_out_name],
                               color=line_color, edgecolor="none", alpha=0.05) #  .rolling(window=10).mean()
        
        if 0:
            edge_colors = cmap(np.linspace(0, 1, len(var_plot[model_out_name])))
            sct         = ax[1].scatter(var_plot['VPD'], var_plot[model_out_name],  color='none', edgecolors=edge_colors,  s=9,
                            alpha=0.05, cmap=cmap, label=model_out_name) #edgecolor='none', c='red'
        
    ax[1].legend(fontsize=8,frameon=False)
    fig.savefig("/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/plots/"+var_name+'_VPD_all_sites'+message,bbox_inches='tight',dpi=300)


## Start plotting

In [4]:
%%time
for i in np.arange(0.0,1.,0.1):    
    print('i=',i)
    bin_vals   = [0+i,0.1+i]
    if len(bin_vals) == 1:
        message    = f'_bin_by_model_EF_{int(bin_vals[0]*10)}'
    else:
        message    = f'_bin_by_model_EF_{int(bin_vals[0]*10)}_{int(bin_vals[1]*10)}'

    day_time       = False
    if day_time:
        message    = message+'_daytime'
    plot_spatial_land_days(var_name, site_names, PLUMBER2_met_path, bin_by, bin_vals, message, day_time=day_time)

    day_time       = True
    if day_time:
        message    = message+'_daytime'
    plot_spatial_land_days(var_name, site_names, PLUMBER2_met_path, bin_by, bin_vals, message, day_time=day_time)


i= 0.0


NameError: name 'plot_spatial_land_days' is not defined